In [19]:
import numpy as np
import pandas as pd
import json
import re
import string

from nltk.corpus import stopwords

In [2]:
def to_dict(string):
    if string != "[]":
        string = json.loads(string.replace("'", "\""))
        return ",".join([s["screen_name"] for s in string])
    return ""

def to_list(list_):
    if list_ != "[]":
        list_ = list_[1:-1]
        list_ = list_.split(",")
        return ",".join([s.strip().strip("'") for s in list_])
    return ""

def normalize(s):
    replacements = (("á", "a"), ("é", "e"), ("í", "i"), ("ó", "o"), ("ú", "u"))
    for a, b in replacements:
        s = s.lower()
        s = s.replace(a, b)
    return s

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r"", text)

def cleanTxt(text):
    text = re.sub(r"@[a-zA-Z0-9]+", "", text) #Removes @mentions
    text = re.sub(r"#", "", text) #Removing the "#" symbol
    text = re.sub(r"RT[\s]+", "", text) #Removing RT
    text = re.sub(r"https?:\/\/\S+", "", text) #Remove the hyperlink
    return text

def replace_punct(s):
    for i in string.punctuation:
        if i in s:
            s = s.replace(i, "").strip()
    return s

def replace_num(s):
    for i in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
        s = s.replace(i, "")
    return s

def tokenizador(text):
    important_words = []
    for word in text.split(" "):
        if word not in stopwords.words("spanish"):
            if word != "":
                important_words.append(word)
    return " ".join(important_words).strip()

def foo(text):
    forbidden = ("?", "¿", "¡", "!", ",", ".", ";", ":", "-", "'", "+", "$", "/", "*",'«','»', "~", "(", ")")
    aux = ""
    for v in text:
        if not v in forbidden:
            aux += v
    return aux

def quita_palabras_pequeñas(text):
    return " ".join([word for word in text.split(" ") if len(word) > 2])  

In [3]:
df_1 = pd.read_csv("C:/Users/Daniel/Desktop/csv/no_trending.csv")

C:\Users\Daniel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_2 = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/no trends/24_notrending.csv")

C:\Users\Daniel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,20,21,22,23,24,26,28,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_3 = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/no trends/el_carlos_24_25.csv")

In [6]:
df_4 = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/no trends/el_ismael_24_25.csv")

In [7]:
df_5 = pd.read_csv("C:/Users/Daniel/Desktop/csv/dia 24/no trends/no_trending_dia_24.csv", sep = ";")

In [8]:
df = pd.concat([df_1, df_2, df_3, df_4, df_5])

In [9]:
df.shape

(6487098, 41)

In [10]:
df.drop(df.columns[0], axis = 1, inplace = True)

In [11]:
df.shape

(6487098, 40)

In [12]:
df.drop("Unnamed: 0.1", axis = 1, inplace = True)

In [13]:
columns_to_drop = ["conversation_id", "cashtags", "timezone", "user_id", "name", "near", "geo", "source",
                   "user_rt_id", "user_rt", "retweet_id", "retweet_date", "translate", "trans_src",
                   "trans_dest", "place", "quote_url", "thumbnail", "created_at", "id", "link"]

df.drop(columns_to_drop, axis = 1, inplace = True)

df = df[df.language == "es"]

df.drop("language", axis = 1, inplace = True)

df = df.reset_index()
df.drop(df.columns[0], axis = 1)

,date,time,username,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,retweet,video,reply_to,hours,day,month
0,2020-02-24,23:59:59,germanlorenz21,"@diegojokas Que hdp que sos Jokas,jajaja.Gol e...",[],[],[],0.0,0.0,0.0,[],False,0.0,"[{'screen_name': 'diegojokas', 'name': 'diego ...",NaN,NaN,NaN
1,2020-02-24,23:59:59,bicharocio,"@gusvalfi Yo lo viví, mi mamá era así, y nosot...",[],[],[],1.0,1.0,2.0,[],False,0.0,"[{'screen_name': 'gusvalfi', 'name': 'Gustavo ...",NaN,NaN,NaN
2,2020-02-24,23:59:59,ruddypujolscasa,@MeltonPinedaF Y el pueblo le entró al PLD que...,[],[],[],0.0,0.0,2.0,[],False,0.0,"[{'screen_name': 'MeltonPinedaF', 'name': 'Mel...",NaN,NaN,NaN
3,2020-02-24,23:59:59,carlosh29,@CalleLaJeta_ llegan y los sacan en carro de ...,[],[],[],1.0,0.0,3.0,[],False,0.0,"[{'screen_name': 'CalleLaJeta_', 'name': 'Fred...",NaN,NaN,NaN
4,2020-02-24,23:59:59,fer_ortiz17,Sabían que se debe llamar realmente mejor amig...,[],[],[],1.0,0.0,4.0,[],False,0.0,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921317,2020-02-24,16:48:44,jaguiilarr,"@JessicaGzS Esque no es de que te lo den o no,...",[],[],[],0.0,0.0,3.0,['enopiniones'],False,0.0,"[{'screen_name': 'JessicaGzS', 'name': 'JessiK...",NaN,NaN,NaN
2921318,2020-02-24,16:48:44,lxivpollo,@FabianPrl Numeros logicos y acordes a que un ...,[],[],[],0.0,0.0,4.0,[],False,0.0,"[{'screen_name': 'FabianPrl', 'name': 'Prol. H...",NaN,NaN,NaN
2921319,2020-02-24,16:48:44,raulsoriamn,@PaleRock @diegosinhue El Gober Sinhué es un i...,[],[],[],0.0,0.0,1.0,[],False,0.0,"[{'screen_name': 'PaleRock', 'name': 'Darius',...",NaN,NaN,NaN
2921320,2020-02-24,16:48:44,erickmono2,@chicacortazar Ya tienes el pasaporte?,[],[],[],0.0,0.0,1.0,[],False,0.0,"[{'screen_name': 'chicacortazar', 'name': 'Eli...",NaN,NaN,NaN


In [14]:
reply_to_rows = []
for num, row in enumerate(df.reply_to):
    try:
        to_dict(row)
    except:
        reply_to_rows.append(num)
        
df.drop(reply_to_rows, inplace = True)

df.reply_to = df.reply_to.apply(to_dict)

df = df.reset_index().drop("index", axis = 1)

mention_rows = []
for num, row in enumerate(df.mentions):
    try:
        to_dict(row)
    except:
        mention_rows.append(num)
        
df.drop(mention_rows, inplace = True)

df.mentions = df.mentions.apply(to_dict)

df = df.reset_index().drop("index", axis = 1)

hashtags_rows = []
for num, row in enumerate(df.hashtags):
    try:
        to_list(row)
    except:
        hashtags_rows.append(num)
        
df.drop(hashtags_rows, inplace = True)

df.hashtags = df.hashtags.apply(to_list)

df = df.reset_index().drop("index", axis = 1)

In [15]:
df.photos = df.photos.apply(lambda x : 1 if x != "[]" else 0)
df.retweet = df.retweet.apply(lambda x : 1 if x == "True" else 0)
df.urls = df.urls.apply(lambda x : 1 if x != "[]" else 0)

In [ ]:
%%time
df.tweet = df.tweet.apply(normalize)
df.tweet = df.tweet.apply(deEmojify)
df.tweet = df.tweet.apply(cleanTxt)
df.tweet = df.tweet.apply(replace_punct)
df.tweet = df.tweet.apply(replace_num)

df.tweet = df.tweet.apply(tokenizador)
df.tweet = df.tweet.apply(foo)
df.tweet = df.tweet.apply(quita_palabras_pequeñas)

In [18]:
df["month"] = df.date.apply(lambda x : x[5 : 7])
df["day"] = df.date.apply(lambda x : x[-2:])

df["hour"] = df.time.apply(lambda x : x[:2])
df["minute"] = df.time.apply(lambda x : x[3:5])
df["second"] = df.time.apply(lambda x : x[6:])

In [ ]:
df["mentions_count"] = [len(mention.split(",")) if type(mention) == str else 0 for mention in df.mentions]

df["reply_to_count"] = [len(reply.split(",")) if type(reply) == str else 0 for reply in df.reply_to]

df["hashtags_count"] =  [len(hashtag.split(",")) if type(hashtag) == str else 0 for hashtag in df.hashtags]